In [ ]:
!pip install transformers

In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline
from sklearn.model_selection import train_test_split
from nltk.corpus import movie_reviews
from sklearn.metrics import accuracy_score

In [ ]:
# Load the NLTK movie review dataset
positive_reviews = [movie_reviews.raw(fileid) for fileid in movie_reviews.fileids('pos')]
negative_reviews = [movie_reviews.raw(fileid) for fileid in movie_reviews.fileids('neg')]
reviews = positive_reviews + negative_reviews
labels = [1] * len(positive_reviews) + [0] * len(negative_reviews)

# Split the data into training and testing sets
train_reviews, test_reviews, train_labels, test_labels = train_test_split(reviews, labels, test_size=0.2, random_state=42)

LookupError: ignored

In [ ]:
# Load the BERT model and tokenizer
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)

# Tokenize and prepare the data
train_encodings = tokenizer(train_reviews, truncation=True, padding=True, return_tensors='pt')
test_encodings = tokenizer(test_reviews, truncation=True, padding=True, return_tensors='pt')

# Create PyTorch datasets
train_dataset = torch.utils.data.TensorDataset(train_encodings.input_ids, train_encodings.attention_mask, torch.tensor(train_labels))
test_dataset = torch.utils.data.TensorDataset(test_encodings.input_ids, test_encodings.attention_mask, torch.tensor(test_labels))

# Create data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8, shuffle=False)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Define a function to calculate accuracy
def get_accuracy(predictions, labels):
    return accuracy_score(labels, predictions)

# Create a sentiment analysis pipeline
sentiment_analyzer = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)

In [ ]:
# Predict sentiment on user input
user_input = input("Enter the chat: ")
user_prediction = sentiment_analyzer(user_input)[0]
if user_prediction['label'] == 'LABEL_1':
    print(f"Sentiment: Positive (score: {user_prediction['score']:.2f})")
else:
    print(f"Sentiment: Negative (score: {1 - user_prediction['score']:.2f})")

# Test the model on the test data
model.eval()
predictions = []
true_labels = []

Enter the chat: you are so good
Sentiment: Positive (score: 0.59)


In [ ]:
with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, labels = batch
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predicted_labels = torch.argmax(logits, dim=1)
        predictions.extend(predicted_labels.tolist())
        true_labels.extend(labels.tolist())

accuracy = get_accuracy(predictions, true_labels)
print(f"Accuracy on test data: {accuracy * 100:.2f}%")

Accuracy on test data: 49.75%
